[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/petro1eum/trust_chain/blob/main/examples/trustchain_full_demo.ipynb)

# TrustChain Full Demo with Real LLM

This notebook tests **all TrustChain features** with a real LLM API (OpenRouter).

**Features tested:**
1. Basic signing & verification
2. Chain of Trust (linked operations)
3. Multi-tool agent workflow
4. Merkle audit trail
5. Tamper detection
6. Audit export (JSON/HTML)
7. Multi-tenant isolation

In [ ]:
# Install dependencies
!pip install -q trustchain openai python-dotenv

In [ ]:
# Load API key from environment or .env file
import os

from dotenv import load_dotenv

load_dotenv()  # Loads from .env file if present

# For Colab: set your key here (or use Colab secrets)
# os.environ['OPENROUTER_API_KEY'] = 'sk-or-v1-...'  # Uncomment if needed

API_KEY = os.environ.get('OPENROUTER_API_KEY')
if API_KEY:
    print(f'✅ API key loaded: {API_KEY[:12]}...{API_KEY[-4:]}')
else:
    print('⚠️ No API key found. Set OPENROUTER_API_KEY environment variable.')

In [ ]:
# Setup
import copy
import json
import tempfile

from openai import OpenAI

from trustchain import TrustChain, TrustChainConfig
from trustchain.v2.merkle import MerkleTree, verify_proof
from trustchain.v2.tenants import TenantManager

# OpenRouter client
client = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=API_KEY
)

# TrustChain instance
tc = TrustChain(TrustChainConfig(enable_nonce=False))

print('✅ TrustChain and OpenRouter client ready')
print(f'🔑 Key ID: {tc.get_key_id()}')

---
## 1️⃣ Basic Signing & Verification

In [ ]:
# Sign data directly
signed = tc._signer.sign(
    tool_id='test_tool',
    data={'message': 'Hello TrustChain!'}
)

print('📝 Signed response:')
print(f'   Tool: {signed.tool_id}')
print(f'   Data: {signed.data}')
print(f'   Signature: {signed.signature[:40]}...')
print(f'   Timestamp: {signed.timestamp}')
print(f'\n✅ Verified: {tc.verify(signed)}')

---
## 2️⃣ Chain of Trust with Real LLM

In [ ]:
# Define tools for the LLM
tools = [
    {'type': 'function', 'function': {'name': 'get_weather', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}}}},
    {'type': 'function', 'function': {'name': 'calculate', 'parameters': {'type': 'object', 'properties': {'expr': {'type': 'string'}}}}},
    {'type': 'function', 'function': {'name': 'search', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}}}}
]

# Tool implementations
def execute_tool(name, args):
    if name == 'get_weather':
        return f"Sunny, 22°C in {args.get('city', 'Unknown')}"
    elif name == 'calculate':
        return str(eval(args.get('expr', '0')))
    elif name == 'search':
        return f"Found 5 results for: {args.get('query', '')}"
    return 'Unknown'

# Multi-step agent workflow
chain = []
queries = [
    'What is the weather in Tokyo? Use the tool.',
    'Calculate 2^10 using the calculate tool.',
    'Search for AI security using the search tool.'
]

print('🤖 Running multi-step agent workflow...\n')

for i, query in enumerate(queries, 1):
    print(f'📤 Step {i}: {query}')

    response = client.chat.completions.create(
        model='anthropic/claude-3-haiku',
        messages=[{'role': 'user', 'content': query}],
        tools=tools,
        tool_choice='auto',
        max_tokens=100
    )

    for tc_call in response.choices[0].message.tool_calls or []:
        args = json.loads(tc_call.function.arguments)
        result = execute_tool(tc_call.function.name, args)

        # Sign with parent for chain of trust
        parent_sig = chain[-1].signature if chain else None
        signed = tc._signer.sign(
            tool_id=tc_call.function.name,
            data={'args': args, 'result': result},
            parent_signature=parent_sig
        )
        chain.append(signed)

        print(f'   ✅ {tc_call.function.name}: {result}')
        print(f'   🔏 Signature: {signed.signature[:32]}...')
        if parent_sig:
            print(f'   ⛓️ Linked to parent: {parent_sig[:32]}...')
    print()

print(f'📊 Chain complete: {len(chain)} tool calls')
print(f'✅ All signatures valid: {all(tc.verify(s) for s in chain)}')

---
## 3️⃣ Merkle Audit Trail

In [ ]:
# Build Merkle tree from chain signatures
signatures = [s.signature for s in chain]
tree = MerkleTree.from_chunks(signatures)

print('🌳 Merkle Tree Audit Trail')
print(f'   Root: {tree.root[:48]}...')
print(f'   Leaves: {len(tree.leaves)}')

# Verify inclusion proof
proof = tree.get_proof(0)
valid = verify_proof(signatures[0], proof, tree.root)
print('\n🔍 Verifying signature[0] is in tree...')
print(f'   Proof length: {len(proof.proof) if hasattr(proof, "proof") else "N/A"}')
print(f'   ✅ Verified: {valid}')

---
## 4️⃣ Tamper Detection

In [ ]:
# Copy original and tamper with it
original = chain[0]
tampered = copy.deepcopy(original)
tampered.data['result'] = 'HACKED: 999999 BTC'

print('🛡️ Tamper Detection Test')
print(f'\n   Original data: {original.data}')
print(f'   Tampered data: {tampered.data}')
print(f'\n   Original verified: {tc.verify(original)} ✅')
print(f'   Tampered verified: {tc.verify(tampered)} ❌')

---
## 5️⃣ Multi-Tenant Isolation

In [ ]:
with tempfile.TemporaryDirectory() as key_dir:
    # Create tenant manager
    manager = TenantManager(key_storage_dir=key_dir)

    # Create separate tenants
    tenants = {
        name: manager.get_or_create(name)
        for name in ['company_a', 'company_b', 'company_c']
    }

    print('🏢 Multi-Tenant Isolation Test')
    for name, tenant_tc in tenants.items():
        print(f'   {name}: Key {tenant_tc.get_key_id()[:20]}...')

    # Sign data with tenant A
    data_a = tenants['company_a']._signer.sign('order', {'id': 12345, 'amount': 100})
    print('\n📝 Company A signed order #12345')

    # Try to verify with tenant B (should fail)
    can_b_verify_a = tenants['company_b'].verify(data_a)
    print(f'   Company B can verify A\'s data: {can_b_verify_a} ❌')

    # Original tenant can verify (should pass)
    can_a_verify_a = tenants['company_a'].verify(data_a)
    print(f'   Company A can verify own data: {can_a_verify_a} ✅')

---
## 6️⃣ Export Audit Trail

In [ ]:
from trustchain.ui.explorer import ChainExplorer

# Create explorer from chain
explorer = ChainExplorer(responses=chain, tc=tc)

# Export JSON
json_export = explorer.to_json()
parsed = json.loads(json_export)

print('📤 Audit Trail Export')
print(f'   Entries: {len(parsed)}')
print('\n📄 JSON Preview:')
for entry in parsed:
    print(f'   - {entry["tool_id"]}: sig={entry["signature"][:20]}...')

# Export HTML
html_path = explorer.export_html('audit_report.html')
print(f'\n📊 HTML report saved: {html_path}')

---
## ✅ Summary

All TrustChain features verified with real LLM (Claude 3 Haiku via OpenRouter):

| Feature | Status |
|---------|--------|
| Basic signing | ✅ |
| Verification | ✅ |
| Chain of Trust | ✅ |
| Real LLM tool calls | ✅ |
| Merkle audit trail | ✅ |
| Tamper detection | ✅ |
| Multi-tenant isolation | ✅ |
| Audit export (JSON/HTML) | ✅ |